<a href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/tutorial/skelton_back_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RustyBot(rbot)のフレームワークスケルトン(BackTest)

フレームワークで動かすためのBotの構造サンプルを提供します。

`on_init`, `on_tick`, `on_clock`, `on_update`を必要に応じて実装します。

バックテストはColabで動きます。試してみてください。


参考：
forward, real_runはWebSocketが繋げるローカル環境が必要です。
* [forward test](./skelton_forward_test.ipynb)
* [real run](./skelton_real_run_testnet.ipynb)

In [2]:

class SkeltonAgent:      # クラス名は任意です
    def __init__(self):
        """Botの初期化処理です。パラメータなどを設定するといいでしょう。利用しなくても構いません。
        """
        self.tick_count = 0         # on_tickが呼び出された回数をカウントします。

    def on_init(self, session):
        """Botの初期化処理。Botの初期化時に一度だけ呼ばれます。
        通常はsession.clock_interval_secを指定しon_clockの呼び出し間隔を設定します。
        Args:
            session: セッション情報（Botの初期化時用に渡されます）
        """
        session.clock_interval_sec = 60 * 60 * 1        # 1時間ごとにon_clockを呼び出す


    def on_tick(self, session, side, price, size):
        """取引所からの全ての約定イベント毎に呼び出される処理です（高頻度で呼び出されます）
        Args:
            session: セッション情報（市況情報の取得や注文するために利用します)
            side: 売買区分です。"Buy"または"Sell"が設定されます。
            price: 約定価格です。
            size: 約定数量です。
        """

        # on_tickは高頻度によびだされるので、１万回に1回だけ内容をプリントします。
        if self.tick_count % 10_000 == 0:
            print("on_tick: ", side, price, size)

        self.tick_count += 1

    def on_clock(self, session, clock):
        """定期的にフレームワークから呼び出される処理です。
        session.clock_interval_secで指定した間隔で呼び出されます。

        Args:
            session: セッション情報（市況情報の取得や注文するために利用します)
            clock: 現在時刻です。エポック時間からのマイクロ秒で表されます。
        """
        # 現在の時刻をプリントします。
        print("on_clock: ", clock, ": ", time_string(clock))

    def on_update(self, session, updated_order):
        """自分の注文状態が変化した場合に呼び出される処理です。
        Args:
            session: セッション情報（市況情報の取得や注文するために利用します)
            session: セッション情報（市況情報の取得や注文するために利用します)
            updated_order: 注文状態が変化した注文情報です。
        """
        # 注文状態が変化した注文情報をプリントします。オーダーを発行しない限り呼び出されません。
        print("on_update", updated_order)



## バックテストの準備

In [3]:
# rbotをインストールします。
! pip install --upgrade pip
! pip install rbot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 15.8 MB/s eta 0:00:00


In [4]:
# Binanceマーケットを指定します。

from rbot import BinanceMarket
from rbot import BinanceConfig
from rbot import time_string

market = BinanceMarket(BinanceConfig.BTCUSDT)

rbot version:  0.2.13
!!! ABSOLUTELY NO WARRANTY !!!
!!!  USE AT YOUR OWN RISK  !!!
See document at https://github.com/yasstake/rusty-bot
 All rights reserved. (c) 2022-2023 rbot developers / yasstake


In [5]:
# 過去ログを１日分ダウンロードします。

market.download(ndays=1, verbose=True, archive_only=True)

# 更新系のオペレーションは１つのプロセスしかできません。他にプロセスがある場合はデッドロックする場合があります。
# また大量のファイルをダウンロードするため、ディスク容量に注意してください。


1741646

In [6]:
# DBの場所は以下の場所にあります。古いバージョンのrbotと非互換が発生した場合などDBファイルを一旦削除してください。
market.file_name

'/root/.local/share/rbot/DB/BN/SPOT-BTCUSDT.db'

In [7]:
from rbot import Runner
from rbot import NOW, DAYS

agent = SkeltonAgent()
runner = Runner()

session = runner.back_test(
                market=market,
                agent=agent,
                start_time=NOW() - DAYS(1),
                end_time=0,
                verbose=True
            )

session

on_clock:  1702890000000000 :  2023-12-18T09:00:00.000000
on_tick:  Buy 40987.96 0.00244
on_tick:  Buy 41080.0 0.00117
on_tick:  Buy 41063.99 0.17856
on_clock:  1702893600000000 :  2023-12-18T10:00:00.000000
on_tick:  Buy 40958.37 0.00206
on_tick:  Sell 40898.81 0.00425
on_tick:  Sell 40938.8 0.00015
on_tick:  Buy 40813.77 0.02061
on_tick:  Buy 40808.27 0.16244
on_tick:  Buy 40711.1 0.0006
on_tick:  Sell 40668.0 1.13462
on_tick:  Sell 40619.0 0.0002
on_tick:  Sell 40665.17 0.00082
on_tick:  Sell 40751.1 0.00268
on_tick:  Sell 40758.15 0.00148
on_clock:  1702897200000000 :  2023-12-18T11:00:00.000000
on_tick:  Sell 40824.01 0.01251
on_tick:  Buy 40905.64 0.00019
on_tick:  Buy 41112.53 0.00652
on_tick:  Buy 41121.1 0.06307
on_tick:  Buy 41131.09 0.00315
on_tick:  Sell 41023.86 0.06426
on_tick:  Sell 40980.8 0.00273
on_tick:  Buy 41013.98 0.5
on_clock:  1702900800000000 :  2023-12-18T12:00:00.000000
on_tick:  Sell 41107.12 0.01125
on_tick:  Sell 41049.98 0.24383
on_tick:  Sell 40978.0 0.0

{"timestamp":1702943999999000,"timestamp_str": 2023-12-18T23:59:59.999000,
"orders":{"buy":{"asc":false,"list":[]}, "sell":{"asc":true,"list":[]}}, 
"account":{"home":"0.0","home_free":"0.0","home_locked":"0.0","foreign":"0.0","foreign_free":"0.0","foreign_locked":"0.0"}, "psudo_account":{"home":"0.0","home_free":"0.0","home_locked":"0.0","foreign":"0.0","foreign_free":"0.0","foreign_locked":"0.0"},"psudo_position":0.0}

In [11]:
# 実行したセッションからログオブジェクトを取得して分析します。
# 今回はAgentがなにもしていないのでログがありませんが、
# 以下流れを示します。
log = session.log

In [9]:
# 今回はオーダー発行していないので出力されません。
log.orders

log_id,symbol,update_time,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change,open_position,close_position,position,profit,fee,total_profit,sum_profit
i64,str,datetime[μs],datetime[μs],str,str,str,str,str,f64,f64,f64,str,f64,f64,f64,f64,str,bool,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64


In [10]:
# 今回はインジケーターを保存していないので出力されません。
# Agent中で session.log_indicator("indicator", 1.0)
# のように保存すると、保存時刻と値がDataFrameで取得できます。
# キーとなる文字列は任意のものが任意数利用できます。
log['indicators']

timestamp,indicators
datetime[μs],f64
